# Challenge 03-C-Embedding 

## 1. Overview 

In the last challenge (03-B-Chunking), we worked towards understanding token limits with LLM and utilizing chunking. Now if there are gigabytes of data, we will have lots of chunks to be created as well. Is there a way to select the most relevant chunks of text? The answer is yes. To solve this problem, we can take a look at a process called Embedding. Embedding helps us create numerical representations for all the chunks. Then, we can find the most similar chunks in the the list of embeddings. One popular way to find the similar chunks is through cosine similarity.

### **Embeddings Overview**
An embedding is a special format of data representation that can be easily utilized by machine learning models and algorithms. The embedding is an information dense representation of the semantic meaning of a piece of text. Each embedding is a vector of floating-point numbers, such that the distance between two embeddings in the vector space is correlated with semantic similarity between two inputs in the original format. For example, if two texts are similar, then their vector representations should also be similar.

Different Azure OpenAI embedding models are specifically created to be good at particular tasks:
- Similarity embeddings are good at capturing semantic similarity between two or more pieces of text.
- Text search embeddings help find which long document is relevant to a short query.
- Code search embeddings are useful for embedding code snippets and embedding nature language search queries.

Embeddings make it easier to do machine learning on large inputs representing words by capturing the semantic similarities in a vector space. Therefore, we can use embeddings to if two text chunks are semantically related or similar, and inherently provide a score to assess similarity.

### **Cosine Similarity**
A previously used approach to match similar documents was based on counting maximum number of common words between documents. This is flawed since as the document size increases, the overlap of common words increases even if the topics differ. Therefore cosine similarity is a better approach.

Mathematically, cosine similarity measures the cosine of the angle between two vectors projected in a multi-dimensional space. This is beneficial because if two documents are far apart by Euclidean distance because of size, they could still have a smaller angle between them and therefore higher cosine similarity.

The Azure OpenAI embeddings rely on cosine similarity to compute similarity between documents and a query.

### **Applications**

Embeddings can be created for all different data types including images, audio, video, and text. In this notebook, we will look at generating embeddings for text and csv files. 

There are many applications in which embeddings can be useful. For example, let's say you want to classify a piece of text. Once embeddings are generated, they can be inserted into a machine learning model to predict the right label. In addition, you can utilize embeddings for similarity in time series data, graph data, or for user profile or products. A very popular use case is one that involves semantic search. If you want to retrieve documents that are very relevant to your query, embeddings can be generated for both the query as well as the documents in order to get an accurate response. We will see an example of this in Challenge 4.

## 2. Let's Start Implementation

You will need to import the needed modules. The following cells are key setup steps you completed in the previous challenges.

In [1]:
! pip install num2words
! pip install plotly
! pip install "openai==0.28.1" 

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 2.1 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 0.27.2
    Uninstalling openai-0.27.2:
      Successfully uninstalled openai-0.27.2


In [2]:
import openai
import os
import re 
import requests
import sys
from num2words import num2words 
import pandas as pd 
import numpy as np
from openai.embeddings_utils import get_embedding, cosine_similarity 
import tiktoken
from dotenv import load_dotenv
from tenacity import retry, wait_random_exponential, stop_after_attempt
load_dotenv() 

True

Set up your environment to access your Azure OpenAI keys. Refer to your Azure OpenAI resource in the Azure Portal to retrieve information regarding your Azure OpenAI endpoint and keys. 

For security purposes, store your sensitive information in an .env file.

In [3]:
openai.api_type = os.getenv("OPENAI_API_TYPE")
openai.api_key = os.environ.get("OPENAI_API_KEY")
openai.api_base = os.environ.get("OPENAI_API_BASE")
openai.api_version = os.getenv("OPENAI_API_VERSION")
embedding_model=os.getenv("EMBEDDING_MODEL_NAME")

## 3. Generate Embeddings on text

#### Student Task #1:
Use the Azure OpenAI Embeddings class to create an embedding for the input text below. 

In [4]:

input="I would like to order a pizza"

# Add code here
embedding = openai.Embedding.create(
    input="I would like to order a pizza", engine=embedding_model
)["data"][0]["embedding"]
print(embedding)
len(embedding)

[0.006304064299911261, -0.013728577643632889, -0.013703952543437481, -0.013322261162102222, -0.020143454894423485, 0.008280240930616856, -0.012035591527819633, -0.005177459213882685, 0.0072521367110311985, -0.017976433038711548, 0.034204475581645966, -0.005823872052133083, 0.004764986224472523, -0.008421835489571095, 0.009825475513935089, -0.006359471008181572, 0.02994430623948574, -0.009726974181830883, 0.011327615939080715, -0.007707703858613968, 0.00011196793639101088, -0.013777827844023705, 0.01578478515148163, -0.008680401369929314, -0.016819046810269356, 0.002884232671931386, 0.004047775641083717, 0.01088436134159565, -0.004571062047034502, -0.004487952217459679, 0.042281556874513626, 0.015193779952824116, -0.017373114824295044, -0.025203945115208626, -0.017816368490457535, 0.0024425170850008726, -0.0027826533187180758, -0.010213322937488556, -0.002467142418026924, -0.028762293979525566, 0.018234997987747192, 0.0039738998748362064, 0.004897346720099449, -0.036987125873565674, -0.

1536

The openai.Embedding.create() method will take a list of text - here we have a single sentence - and then will return a list containing a single embedding. You can use these embeddings when searching, providing recommendations, classification, and more.

### 3.1 Generate Embeddings for a CSV file

#### Student Task #2:
Enter in the path of the `Automobile.csv` file which you can find in the `/data` folder. Run the cells below.

In [6]:
df=pd.read_csv(os.path.join(os.getcwd(),r'../data/Automobile.csv'))
df

,name,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin
0,chevrolet chevelle malibu,18.0,8,307.0,130.0,3504,12.0,70,usa
1,buick skylark 320,15.0,8,350.0,165.0,3693,11.5,70,usa
2,plymouth satellite,18.0,8,318.0,150.0,3436,11.0,70,usa
3,amc rebel sst,16.0,8,304.0,150.0,3433,12.0,70,usa
4,ford torino,17.0,8,302.0,140.0,3449,10.5,70,usa
...,...,...,...,...,...,...,...,...,...
393,ford mustang gl,27.0,4,140.0,86.0,2790,15.6,82,usa
394,vw pickup,44.0,4,97.0,52.0,2130,24.6,82,europe
395,dodge rampage,32.0,4,135.0,84.0,2295,11.6,82,usa
396,ford ranger,28.0,4,120.0,79.0,2625,18.6,82,usa


In [7]:
shortened_df = df[['name', 'mpg', 'origin']]
shortened_df

,name,mpg,origin
0,chevrolet chevelle malibu,18.0,usa
1,buick skylark 320,15.0,usa
2,plymouth satellite,18.0,usa
3,amc rebel sst,16.0,usa
4,ford torino,17.0,usa
...,...,...,...
393,ford mustang gl,27.0,usa
394,vw pickup,44.0,europe
395,dodge rampage,32.0,usa
396,ford ranger,28.0,usa


In [11]:
tokenizer = tiktoken.get_encoding("cl100k_base")
shortened_df['n_tokens'] = shortened_df["name"].apply(lambda x: len(tokenizer.encode(x)))
shortened_df = shortened_df[shortened_df.n_tokens<8192]
len(shortened_df)

398

In [12]:
shortened_df

,name,mpg,origin,n_tokens
0,chevrolet chevelle malibu,18.0,usa,6
1,buick skylark 320,15.0,usa,7
2,plymouth satellite,18.0,usa,3
3,amc rebel sst,16.0,usa,5
4,ford torino,17.0,usa,2
...,...,...,...,...
393,ford mustang gl,27.0,usa,4
394,vw pickup,44.0,europe,2
395,dodge rampage,32.0,usa,3
396,ford ranger,28.0,usa,2


In [13]:
sample_encode = tokenizer.encode(shortened_df.name[0]) 
decode = tokenizer.decode_tokens_bytes(sample_encode)
decode

[b'che', b'vrolet', b' che', b'velle', b' mal', b'ibu']

In [14]:
len(decode)
shortened_df['ada-v2'] = shortened_df['name'].apply(lambda x : get_embedding(x, engine = embedding_model)) 

In [15]:
shortened_df

,name,mpg,origin,n_tokens,ada-v2
0,chevrolet chevelle malibu,18.0,usa,6,"[-0.0343162938952446, 0.0007342947646975517, 0..."
1,buick skylark 320,15.0,usa,7,"[-0.00984190870076418, -0.0031548449769616127,..."
2,plymouth satellite,18.0,usa,3,"[-0.019997337833046913, 0.016940420493483543, ..."
3,amc rebel sst,16.0,usa,5,"[-0.023981362581253052, -0.00803937017917633, ..."
4,ford torino,17.0,usa,2,"[-0.0385330505669117, -0.02776527777314186, -0..."
...,...,...,...,...,...
393,ford mustang gl,27.0,usa,4,"[-0.04225892201066017, -0.0038522197864949703,..."
394,vw pickup,44.0,europe,2,"[-0.02390824444591999, -0.015644479542970657, ..."
395,dodge rampage,32.0,usa,3,"[-0.024077117443084717, -0.005027948413044214,..."
396,ford ranger,28.0,usa,2,"[-0.014123061671853065, -0.02124483324587345, ..."


In [31]:
# Generate Python Code to plot
chat_model=os.getenv("CHAT_MODEL_NAME")

def get_chat_completion(prompt, model=chat_model):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        engine=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
        max_tokens = 200,
        top_p = 1.0
    )
    return response.choices[0].message["content"]

messages =  f"""
Generate python code to 3D plot {shortened_df} values using plotly
"""
response = get_chat_completion(messages)
print(response)

Here's an example code to generate a 3D scatter plot using Plotly for the given dataset:

```python
import plotly.express as px

fig = px.scatter_3d(df, x='mpg', y='origin', z='n_tokens', color='name')
fig.show()
```

This code will create a 3D scatter plot with the `mpg`, `origin`, and `n_tokens` columns as the x, y, and z axes respectively. The `name` column will be used to color-code the data points. You can customize the plot further by adding titles, labels, and adjusting the plot layout.


In [33]:
import plotly.express as px

df = shortened_df

fig = px.scatter_3d(df, x='n_tokens', y='origin', z='mpg', color='name')
fig.show()



/usr/local/python/3.10.13/lib/python3.10/site-packages/plotly/express/_core.py:1979: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



The embeddings generated from the csv file can be used to perform search. You can calculate the cosine similarity between a query embedding and the embeddings from the csv file. Then you can rank the search results to what is most relevant to the query. We will see an application of embedddings in Challenge 4.

## Success Criteria 

To complete this challenge successfully:

* Show an understanding of embeddings by working with different inputs.